# Problem Statement and Description

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Importing the required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import (confusion_matrix, accuracy_score)
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier, GradientBoostingClassifier, RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.svm import SVC

In [3]:
# reading the required csv files
train = pd.read_csv('train_data.csv', na_values = ['$', '#', 'Null', '?'])
test = pd.read_csv('test_data.csv', na_values = ['$', '#', 'Null', '?'])
prices = pd.read_csv('product_prices.csv', na_values = ['$', '#', 'Null', '?'])
date_week_id = pd.read_csv('date_to_week_id_map.csv', na_values = ['$', '#', 'Null', '?'])

In [4]:
# counting the number of rows and columns in each of the files
print('The shape of the train_data file is ',  train.shape)
print('The shape of the test_data file is ',  test.shape)
print('The shape of the product_prices file is ',  prices.shape)
print('The shape of the date_to_week_id_map file is ',  date_week_id.shape)

The shape of the train_data file is  (395000, 7)
The shape of the test_data file is  (15500, 7)
The shape of the product_prices file is  (59000, 4)
The shape of the date_to_week_id_map file is  (821, 2)


__Observation:__ <br>
- The train_data file has 395000 rows and 7 columns
- The test_data file has 15500 rows and 7 columns
- The product_prices file has 59000 rows and 4 columns
- The date_to_week_id_map file has 821 rows and 2 columns

In [5]:
# veiwing the first five rows of the datasets
display(train.head())

display(test.head())

display(prices.head())

display(date_week_id.head())

,date,product_identifier,department_identifier,category_of_product,outlet,state,sales
0,2012-01-01,74,11,others,111,Maharashtra,0
1,2012-01-01,337,11,others,111,Maharashtra,1
2,2012-01-01,423,12,others,111,Maharashtra,0
3,2012-01-01,432,12,others,111,Maharashtra,0
4,2012-01-01,581,21,fast_moving_consumer_goods,111,Maharashtra,0


,id,date,product_identifier,department_identifier,category_of_product,outlet,state
0,1,2014-03-01,74,11,others,111,Maharashtra
1,2,2014-03-01,337,11,others,111,Maharashtra
2,3,2014-03-01,423,12,others,111,Maharashtra
3,4,2014-03-01,432,12,others,111,Maharashtra
4,5,2014-03-01,581,21,fast_moving_consumer_goods,111,Maharashtra


,outlet,product_identifier,week_id,sell_price
0,111,74,49,2.94
1,111,74,50,2.94
2,111,74,51,2.94
3,111,74,52,2.94
4,111,74,53,2.94


,date,week_id
0,2012-01-01,49
1,2012-01-02,49
2,2012-01-03,49
3,2012-01-04,49
4,2012-01-05,49


In [6]:
# veiwing the last five rows of the datasets
display(train.tail())

display(test.tail())

display(prices.tail())

display(date_week_id.tail())

,date,product_identifier,department_identifier,category_of_product,outlet,state,sales
394995,2014-02-28,2932,33,drinks_and_food,333,Kerala,2
394996,2014-02-28,2935,33,drinks_and_food,333,Kerala,8
394997,2014-02-28,3004,33,drinks_and_food,333,Kerala,0
394998,2014-02-28,3008,33,drinks_and_food,333,Kerala,0
394999,2014-02-28,3021,33,drinks_and_food,333,Kerala,0


,id,date,product_identifier,department_identifier,category_of_product,outlet,state
15495,15496,2014-03-31,2932,33,drinks_and_food,333,Kerala
15496,15497,2014-03-31,2935,33,drinks_and_food,333,Kerala
15497,15498,2014-03-31,3004,33,drinks_and_food,333,Kerala
15498,15499,2014-03-31,3008,33,drinks_and_food,333,Kerala
15499,15500,2014-03-31,3021,33,drinks_and_food,333,Kerala


,outlet,product_identifier,week_id,sell_price
58995,333,3021,162,2.08
58996,333,3021,163,2.08
58997,333,3021,164,2.08
58998,333,3021,165,2.08
58999,333,3021,166,2.08


,date,week_id
816,2014-03-27,165
817,2014-03-28,165
818,2014-03-29,166
819,2014-03-30,166
820,2014-03-31,166


__Observations:__ <br>
- The train_data contains information starting from 2012-01-01 to 2014-02-28, i.e., two year and two months of data
- The test_data contains information from 2014-03-01 to 2014-03-31, i.e., one month of data

In [7]:
# obtaining column names present in each of the files
print('The columns of the train_data file are \n',  train.columns, '\n')
print('The columns of the test_data file are \n',  test.columns, '\n')
print('The columns of the product_prices file are \n',  prices.columns, '\n')
print('The columns of the date_to_week_id_map file are \n',  date_week_id.columns, '\n')

The columns of the train_data file are 
 Index(['date', 'product_identifier', 'department_identifier',
       'category_of_product', 'outlet', 'state', 'sales'],
      dtype='object') 

The columns of the test_data file are 
 Index(['id', 'date', 'product_identifier', 'department_identifier',
       'category_of_product', 'outlet', 'state'],
      dtype='object') 

The columns of the product_prices file are 
 Index(['outlet', 'product_identifier', 'week_id', 'sell_price'], dtype='object') 

The columns of the date_to_week_id_map file are 
 Index(['date', 'week_id'], dtype='object') 



### Merging the files

The file, date_to_week_id_map provides us a week id for each of the dates, the same dates are present in in the train_data and test_data datasets. We merge the files based on date to add another column of week_id to the train_data and test_data datasets.

In [8]:
train1 = pd.merge(train, date_week_id, on = 'date', how = 'inner')
train1.shape

(395000, 8)

In [9]:
test1 = pd.merge(test, date_week_id, on = 'date', how = 'inner')
test1.shape

(15500, 8)

__Observation:__
The number of rows has not increased or decreased with the addition of the new columns.

In [10]:
display(train1.head())
display(train1.tail())

,date,product_identifier,department_identifier,category_of_product,outlet,state,sales,week_id
0,2012-01-01,74,11,others,111,Maharashtra,0,49
1,2012-01-01,337,11,others,111,Maharashtra,1,49
2,2012-01-01,423,12,others,111,Maharashtra,0,49
3,2012-01-01,432,12,others,111,Maharashtra,0,49
4,2012-01-01,581,21,fast_moving_consumer_goods,111,Maharashtra,0,49


,date,product_identifier,department_identifier,category_of_product,outlet,state,sales,week_id
394995,2014-02-28,2932,33,drinks_and_food,333,Kerala,2,161
394996,2014-02-28,2935,33,drinks_and_food,333,Kerala,8,161
394997,2014-02-28,3004,33,drinks_and_food,333,Kerala,0,161
394998,2014-02-28,3008,33,drinks_and_food,333,Kerala,0,161
394999,2014-02-28,3021,33,drinks_and_food,333,Kerala,0,161


In [11]:
display(test1.head())
display(test1.tail())

,id,date,product_identifier,department_identifier,category_of_product,outlet,state,week_id
0,1,2014-03-01,74,11,others,111,Maharashtra,162
1,2,2014-03-01,337,11,others,111,Maharashtra,162
2,3,2014-03-01,423,12,others,111,Maharashtra,162
3,4,2014-03-01,432,12,others,111,Maharashtra,162
4,5,2014-03-01,581,21,fast_moving_consumer_goods,111,Maharashtra,162


,id,date,product_identifier,department_identifier,category_of_product,outlet,state,week_id
15495,15496,2014-03-31,2932,33,drinks_and_food,333,Kerala,166
15496,15497,2014-03-31,2935,33,drinks_and_food,333,Kerala,166
15497,15498,2014-03-31,3004,33,drinks_and_food,333,Kerala,166
15498,15499,2014-03-31,3008,33,drinks_and_food,333,Kerala,166
15499,15500,2014-03-31,3021,33,drinks_and_food,333,Kerala,166


In [12]:
train2 = pd.merge(train1, prices, on = ['week_id', 'outlet', 'product_identifier'], how = 'inner')
train2.shape

(395000, 9)

In [13]:
test2 = pd.merge(test1, prices, on = ['outlet', 'product_identifier', 'week_id',], how = 'inner')
test2.shape

(15500, 9)

__Observation:__
The number of rows has not increased or decreased with the addition of the new columns.

In [14]:
display(train2.head())
display(train2.tail())
display(test2.head())
display(test2.tail())

,date,product_identifier,department_identifier,category_of_product,outlet,state,sales,week_id,sell_price
0,2012-01-01,74,11,others,111,Maharashtra,0,49,2.94
1,2012-01-02,74,11,others,111,Maharashtra,0,49,2.94
2,2012-01-03,74,11,others,111,Maharashtra,1,49,2.94
3,2012-01-04,74,11,others,111,Maharashtra,0,49,2.94
4,2012-01-05,74,11,others,111,Maharashtra,0,49,2.94


,date,product_identifier,department_identifier,category_of_product,outlet,state,sales,week_id,sell_price
394995,2014-02-24,3021,33,drinks_and_food,333,Kerala,0,161,2.08
394996,2014-02-25,3021,33,drinks_and_food,333,Kerala,0,161,2.08
394997,2014-02-26,3021,33,drinks_and_food,333,Kerala,0,161,2.08
394998,2014-02-27,3021,33,drinks_and_food,333,Kerala,0,161,2.08
394999,2014-02-28,3021,33,drinks_and_food,333,Kerala,0,161,2.08


,id,date,product_identifier,department_identifier,category_of_product,outlet,state,week_id,sell_price
0,1,2014-03-01,74,11,others,111,Maharashtra,162,3.43
1,501,2014-03-02,74,11,others,111,Maharashtra,162,3.43
2,1001,2014-03-03,74,11,others,111,Maharashtra,162,3.43
3,1501,2014-03-04,74,11,others,111,Maharashtra,162,3.43
4,2001,2014-03-05,74,11,others,111,Maharashtra,162,3.43


,id,date,product_identifier,department_identifier,category_of_product,outlet,state,week_id,sell_price
15495,14999,2014-03-30,3008,33,drinks_and_food,333,Kerala,166,1.98
15496,15499,2014-03-31,3008,33,drinks_and_food,333,Kerala,166,1.98
15497,14500,2014-03-29,3021,33,drinks_and_food,333,Kerala,166,2.08
15498,15000,2014-03-30,3021,33,drinks_and_food,333,Kerala,166,2.08
15499,15500,2014-03-31,3021,33,drinks_and_food,333,Kerala,166,2.08


In [15]:
test2 = test2.sort_values('id')
test2

,id,date,product_identifier,department_identifier,category_of_product,outlet,state,week_id,sell_price
0,1,2014-03-01,74,11,others,111,Maharashtra,162,3.43
7,2,2014-03-01,337,11,others,111,Maharashtra,162,7.78
14,3,2014-03-01,423,12,others,111,Maharashtra,162,0.97
21,4,2014-03-01,432,12,others,111,Maharashtra,162,4.97
28,5,2014-03-01,581,21,fast_moving_consumer_goods,111,Maharashtra,162,4.88
...,...,...,...,...,...,...,...,...,...
15487,15496,2014-03-31,2932,33,drinks_and_food,333,Kerala,166,2.78
15490,15497,2014-03-31,2935,33,drinks_and_food,333,Kerala,166,0.20
15493,15498,2014-03-31,3004,33,drinks_and_food,333,Kerala,166,2.68
15496,15499,2014-03-31,3008,33,drinks_and_food,333,Kerala,166,1.98


In [16]:
df = test2['id']
df

0            1
7            2
14           3
21           4
28           5
         ...  
15487    15496
15490    15497
15493    15498
15496    15499
15499    15500
Name: id, Length: 15500, dtype: int64

In [17]:
type(df)

pandas.core.series.Series

## Data Pre-processing and Cleaning

### Missing Value treatment

Checking for missing values in the train and test data

In [18]:
# train data
for i in range(0,len(train2.columns)):
    print('The number of missing values in the column', train2.columns[i], 'is', train2.isnull().sum()[i])    
print('\n The total number of missing values in the data set is',train2.isnull().sum().sum())

The number of missing values in the column date is 0
The number of missing values in the column product_identifier is 0
The number of missing values in the column department_identifier is 0
The number of missing values in the column category_of_product is 0
The number of missing values in the column outlet is 0
The number of missing values in the column state is 0
The number of missing values in the column sales is 0
The number of missing values in the column week_id is 0
The number of missing values in the column sell_price is 0

 The total number of missing values in the data set is 0


In [19]:
# test data
for i in range(0,len(test2.columns)):
    print('The number of missing values in the column', test2.columns[i], 'is', test2.isnull().sum()[i])    
print('\n The total number of missing values in the data set is',test2.isnull().sum().sum())

The number of missing values in the column id is 0
The number of missing values in the column date is 0
The number of missing values in the column product_identifier is 0
The number of missing values in the column department_identifier is 0
The number of missing values in the column category_of_product is 0
The number of missing values in the column outlet is 0
The number of missing values in the column state is 0
The number of missing values in the column week_id is 0
The number of missing values in the column sell_price is 0

 The total number of missing values in the data set is 0


__Observations:__
Both the datasets do not contain any missing values, so we do not need to do any missing value treatment or imputations.

### Cheching for unique values

In [20]:
# train data
for c in train2.columns.values:
    print('The number of unique values in {} is {}'.format(c, train2[c].nunique()))

The number of unique values in date is 790
The number of unique values in product_identifier is 50
The number of unique values in department_identifier is 6
The number of unique values in category_of_product is 3
The number of unique values in outlet is 10
The number of unique values in state is 3
The number of unique values in sales is 126
The number of unique values in week_id is 113
The number of unique values in sell_price is 107


In [21]:
# test data
for c in test2.columns.values:
    print('The number of unique values in {} is {}'.format(c, test2[c].nunique()))

The number of unique values in id is 15500
The number of unique values in date is 31
The number of unique values in product_identifier is 50
The number of unique values in department_identifier is 6
The number of unique values in category_of_product is 3
The number of unique values in outlet is 10
The number of unique values in state is 3
The number of unique values in week_id is 5
The number of unique values in sell_price is 51


__Observation__
- The number of unique values in each of the columns in the training data is very less compared to the numbr of data point(rows) in them. So, we do not drop any columns at this stage.
- The number of unique values in id column of the test dataset are 15500 which is the same as the number of datapoints in the column, hence we do not require the column and drop it. 
- The rest of the columns in the test data set need not be dropped

In [22]:
submission = test2['id']
display(submission)

0            1
7            2
14           3
21           4
28           5
         ...  
15487    15496
15490    15497
15493    15498
15496    15499
15499    15500
Name: id, Length: 15500, dtype: int64

In [23]:
test2.drop('id', axis = 1, inplace = True)
test2.head()

,date,product_identifier,department_identifier,category_of_product,outlet,state,week_id,sell_price
0,2014-03-01,74,11,others,111,Maharashtra,162,3.43
7,2014-03-01,337,11,others,111,Maharashtra,162,7.78
14,2014-03-01,423,12,others,111,Maharashtra,162,0.97
21,2014-03-01,432,12,others,111,Maharashtra,162,4.97
28,2014-03-01,581,21,fast_moving_consumer_goods,111,Maharashtra,162,4.88


Finding the datatypes of all the columns

In [24]:
train2.dtypes

date                      object
product_identifier         int64
department_identifier      int64
category_of_product       object
outlet                     int64
state                     object
sales                      int64
week_id                    int64
sell_price               float64
dtype: object

In [25]:
test2.dtypes

date                      object
product_identifier         int64
department_identifier      int64
category_of_product       object
outlet                     int64
state                     object
week_id                    int64
sell_price               float64
dtype: object

In [26]:
test2.drop('date', axis = 1, inplace = True)
train2.drop('date', axis = 1, inplace = True)

### Preparing the data for model building

Separating X and y for the training data set

In [27]:
X = train2.loc[:, train2.columns != 'sales']
y = train2['sales']
print(X.shape)
print(y.shape)

(395000, 7)
(395000,)


Separating the numeric and categorical columns

In [28]:
num_cols = X.select_dtypes(include=['float64','int64']).columns
print(num_cols)
cat_cols = X.select_dtypes(include=['object']).columns
print(cat_cols)

Index(['product_identifier', 'department_identifier', 'outlet', 'week_id',
       'sell_price'],
      dtype='object')
Index(['category_of_product', 'state'], dtype='object')


In [29]:
y.unique()

array([  0,   1,   5,   3,   6,   2,   4,   9,  13,  16,   8,  11,   7,
        12,  19,  10,  18,  32,  28,  23,  54,  22,  27,  14,  17,  47,
        15,  26,  20,  33,  87,  35,  36,  25,  34,  31,  43,  53,  21,
        38,  30,  57,  85, 100,  52,  29,  24,  58,  63,  45,  39,  37,
        42,  72,  80,  40,  98,  56,  70,  50,  46,  51,  86,  66,  79,
        41,  64,  81,  48,  82,  69,  94,  44,  60,  68,  49,  77,  61,
        76, 119,  92,  93,  62, 124,  59,  74,  73, 174,  96,  71, 132,
        55, 101,  78,  75,  65, 105,  67, 120, 150, 173,  83,  95, 170,
       116,  90, 108, 139,  84, 293, 241,  88,  97, 102, 156, 114, 126,
       109, 118, 111,  99, 171, 242, 121, 215,  89], dtype=int64)

In [30]:
numeric_transformer =  Pipeline(steps = [('scalar', StandardScaler())])
categorical_transformer = Pipeline(steps = [('onehot', OneHotEncoder(handle_unknown = 'ignore'))])
preprocessor = ColumnTransformer(transformers = [('num', numeric_transformer, num_cols), 
                                                 ('cat', categorical_transformer, cat_cols)])

In [32]:
RF_pipe = Pipeline(steps = [('pp', preprocessor),('algo', RandomForestRegressor(n_estimators = 100, random_state = 0, 
                                                                                max_depth = 12))])
RF_pipe.fit(X, y)

Pipeline(steps=[('pp',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scalar',
                                                                   StandardScaler())]),
                                                  Index(['product_identifier', 'department_identifier', 'outlet', 'week_id',
       'sell_price'],
      dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  Index(['category_of_product', 'state'], dtype='object'))])),
                ('algo', RandomForestRegressor(max_depth=12, random_state=0))])

In [33]:
y_train_pred = RF_pipe.predict(X)
y_test_pred = RF_pipe.predict(test2)

In [34]:
tub = pd.DataFrame(y_test_pred, columns = ['sales'])

In [35]:
tub.insert(0, 'id', df.values)
tub

,id,sales
0,1,0.193939
1,2,0.802603
2,3,0.109783
3,4,0.071777
4,5,1.055028
...,...,...
15495,15496,1.247834
15496,15497,11.065821
15497,15498,0.649294
15498,15499,1.469016
